## Step1.連結到雲端硬碟資料

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
import pandas as pd

Mounted at /content/gdrive


## Step2.從雲端讀入資料

#### 顏色整理
occurrences per unit time

顏色分級:0 (0,3] (3,7] (7,12] (12,18] (18,25] (25,33] (33,42] (42,100] (100,200] (200,+∞]

#### category & 顏色分級一個2維dataframe


In [ ]:
#讀入顏色
input_color= pd.read_csv('gdrive/My Drive/ML_Cybersecurity/colors.csv')
print(input_color)
print(len(input_color.columns))
print(len(input_color.index))
colors = pd.DataFrame(columns=range(0,len(input_color.columns)),index=range(0,len(input_color.index)))
for i in range(len(input_color.columns)):
  for j in range(len(input_color.index)):
    temp=input_color.iloc[j,i]
    R='0x'+temp[1:3]
    G='0x'+temp[3:5]
    B='0x'+temp[5:7]
    R_num=int(str(R), 16)
    G_num=int(str(G), 16)
    B_num=int(str(B), 16)
    colors[i][j]=R_num,G_num,B_num
print(colors)


          1        2        3        4        5        6        7        8  \
0   #FFFFFF  #FFC1E0  #FFAAD5  #FF95CA  #FF79BC  #FF60AF  #FF359A  #FF0080   
1   #FFFFFF  #FFBFFF  #FFA6FF  #FF8EFF  #FF77FF  #FF44FF  #FF00FF  #E800E8   
2   #FFFFFF  #FFDAC8  #FFCBB3  #FFBD9D  #FFAD86  #FF9D6F  #FF8F59  #FF8040   
3   #FFFFFF  #D3FF93  #CCFF80  #B7FF4A  #A8FF24  #9AFF02  #8CEA00  #82D900   
4   #FFFFFF  #CAFFFF  #BBFFFF  #A6FFFF  #4DFFFF  #00FFFF  #00E3E3  #00CACA   
5   #FFFFFF  #C1FFE4  #ADFEDC  #96FED1  #4EFEB3  #1AFD9C  #02F78E  #02DF82   
6   #FFFFFF  #D6D6AD  #CDCD9A  #C2C287  #B9B973  #AFAF61  #A5A552  #949449   
7   #FFFFFF  #DCB5FF  #D3A4FF  #CA8EFF  #BE77FF  #B15BFF  #9F35FF  #921AFF   
8   #FFFFFF  #FFFF6F  #FFFF37  #F9F900  #E1E100  #C4C400  #A6A600  #8C8C00   
9   #FFFFFF  #FFE66F  #FFE153  #FFDC35  #FFD306  #EAC100  #D9B300  #C6A300   
10  #FFFFFF  #D8D8EB  #C7C7E2  #B8B8DC  #A6A6D2  #9999CC  #8080C0  #7373B9   
11  #FFFFFF  #97CBFF  #84C1FF  #66B3FF  #46A3FF  #2894FF  #0080F

#### 決定常數

In [ ]:
unit_time_number=16 #單位時間的數量
text_file_number=25  #json檔的數量(從1開始，所以要+1)

#### 先載入所有檔案，確定category的所有種類有哪些

In [ ]:
import json
categorys=[]
# Reading JSON data from a file
for i in range(1,text_file_number):
  with open('gdrive/My Drive/ML_Cybersecurity/'+str(i)+'.json') as input_json:
    print(i)
    json_data = json.load(input_json)
    data=pd.DataFrame(json_data["behavior"]['processes'])
    data2=data["calls"][1]
    for i in range(len(data2)):
      temp=data2[i]["category"]
      if temp not in categorys:
        categorys.append(temp)
print(len(categorys))
print(categorys)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
15
['synchronisation', 'exception', 'system', 'file', 'misc', 'process', 'ui', 'resource', 'registry', 'network', '__notification__', 'ole', 'crypto', 'services', 'netapi']


#### 計算api出現次數&產圖

In [ ]:
import json
from PIL import Image
import sys

# Reading JSON data from a file
for file_number in range(1,text_file_number):
  with open('gdrive/My Drive/ML_Cybersecurity/'+str(file_number)+'.json') as input_json:
      json_data = json.load(input_json)
      data=pd.DataFrame(json_data["behavior"]['processes'])# Converting parts of JSON data to a pandas DataFrame

      #獲取開始&結束時間
      data2=data["calls"][1]
      start_time=data2[0]["time"]
      end_time=0

      for i in range(len(data2)):
        if(data2[i]["time"]<start_time):
          start_time=data2[i]["time"]
        if(data2[i]["time"]>end_time):
          end_time=data2[i]["time"]
      #print(start_time)
      #print(end_time)

      #計算單位時間
      unit_time=(end_time-start_time)/unit_time_number

      #統計category & 每單位時間次數2維dataframe
      api_times = pd.DataFrame(np.zeros(unit_time_number*len(categorys)).reshape((len(categorys),unit_time_number)),columns=range(1,unit_time_number+1),index=categorys)
      for i in range(len(data2)):
        temp_time=int((data2[i]['time']-start_time)/unit_time)+1 #確定在第幾個單位時間被call
        if(temp_time==unit_time_number+1):
          temp_time=temp_time-1 #最後一個API算在最後一個單位時間
        api_times.loc[data2[i]['category'],temp_time]=api_times.loc[data2[i]['category'],temp_time]+1  #該API種類單位時間被call次數+1

      print(str(file_number)+"的各類API次數統計")
      print()
      print(api_times)


      #產圖
      outputImage = Image.new('RGB', (len(categorys),unit_time_number))
      for x in range(len(categorys)):
        for m in range(unit_time_number):
          #0 (0,3] (3,7] (7,12] (12,18] (18,25] (25,33] (33,42] (42,100] (100,200] (200,+∞]
            if(api_times.iloc[x,m]==0):
              y=0
            elif((api_times.iloc[x,m]>0) and (api_times.iloc[x,m]<=3)):
              y=1
            elif((api_times.iloc[x,m]>3) and (api_times.iloc[x,m]<=7)):
              y=2
            elif((api_times.iloc[x,m]>7) and (api_times.iloc[x,m]<=12)):
              y=3
            elif((api_times.iloc[x,m]>12) and (api_times.iloc[x,m]<=18)):
              y=4
            elif((api_times.iloc[x,m]>18) and (api_times.iloc[x,m]<=25)):
              y=5
            elif((api_times.iloc[x,m]>25) and (api_times.iloc[x,m]<=33)):
              y=6
            elif((api_times.iloc[x,m]>33) and (api_times.iloc[x,m]<=42)):
              y=7
            elif((api_times.iloc[x,m]>42) and (api_times.iloc[x,m]<=100)):
              y=8
            elif((api_times.iloc[x,m]>100) and (api_times.iloc[x,m]<=200)):
              y=9
            elif(api_times.iloc[x,m]>200):
              y=10
            outputImage.putpixel(((x,m)),colors.iloc[x][y])

      #輸出圖片
      outputImage.save('gdrive/My Drive/ML_Cybersecurity/'+str(file_number)+'.jpg')

1的各類API次數統計

                   1    2    3    4    5    6    7    8    9    10   11   12  \
synchronisation   1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
exception         1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
system            1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  6.0  0.0   
file              2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
misc              0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0  2.0  0.0   
process           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0   
ui                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
resource          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
registry          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
network           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
__notification__  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
ole               0.0  0.0 